In [1]:
import os


dim = 3
float_precision = 'float32'
root_file = os.getcwd()
mesh_path = 'tetrahedron_big.msh'
mesh_path = os.path.join(root_file, 'mesh', mesh_path)



In [3]:
import importlib

module = importlib.import_module("create_domain")
importlib.reload(module)
Domain = getattr(module, "Domain")

domain = Domain(mesh_path, dim)
print(domain.cells.shape)

Init
triangle
tetra
_create_cells
node_cellid
cell_cellnid
_create_info
Execution time: 5.219851 seconds
(5821794, 5)


In [3]:
# 8.5-10s || 46s
#

In [2]:
from manapy.partitions import MeshPartition
from manapy.base.base import Struct
from manapy.ddm import Domain

def create_partitions(mesh_file_path, float_precision, dim):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
  MeshPartition(mesh_file_path, dim=dim, conf=running_conf, periodic=[0,0,0])
  domain = Domain(dim=dim, conf=running_conf)
  return domain

create_partitions(mesh_path, float_precision, dim)



Reading gmsh file ...
Saving partition files ...
Number of Cells: 5821794
Number of Vertices: 1000000
Local domain contruction ...
Execution time: 25.724517 seconds


In [65]:
import numpy as np

def _is_in_array(array: 'int[:]', item: 'int') -> 'int':
  """
    Check if an item is in the array
    Return 1 if the item is in the array otherwise 0

    Note:
      The number of item in the array must be array[-1]
  """
  for i in range(array[-1]):
    if item == array[i]:
      return 1
  return 0

def _create_cell_cellnid_old(
  cells: 'int[:, :]',
  node_cellid: 'int[:, :]',
  cell_cellnid: 'int[:, :]',
):
  """
    Get all neighboring cells by collecting adjacent cells from each node of the cell.
  """

  for i in range(cells.shape[0]):
    for j in range(cells[i][-1]):
      node_n = node_cellid[cells[i][j]]
      for k in range(node_n[-1]):
        if node_n[k] != i and _is_in_array(cell_cellnid[i], node_n[k]) == 0:
          size = cell_cellnid[i][-1]
          cell_cellnid[i][-1] += 1
          cell_cellnid[i][size] = node_n[k]

def _create_cell_cellnid_new(
  cells: 'int[:, :]',
  node_cellid: 'int[:, :]',
  cell_cellnid: 'int[:, :]',
):
  """
    Get all neighboring cells by collecting adjacent cells from each node of the cell.
  """

  for i in range(cells.shape[0]):
    for j in range(cells[i, -1]):
      node_n = node_cellid[cells[i][j]]
      for k in range(node_n[-1]):
        nc = node_n[k]
        size = cell_cellnid[nc, -1]
        if nc != i and (size <= 0 or cell_cellnid[nc, size - 1] != i):
          cell_cellnid[nc, size] = i
          cell_cellnid[nc, -1] += 1

cell_cellnid_old = np.ones(shape=(domain.nb_cells, domain.max_cell_cellnid + 1), dtype=np.int32) * -1
cell_cellnid_new = np.ones(shape=(domain.nb_cells, domain.max_cell_cellnid + 1), dtype=np.int32) * -1
cell_cellnid_old[:, -1] = 0
cell_cellnid_new[:, -1] = 0
_create_cell_cellnid_old(domain.cells, domain.node_cellid, cell_cellnid_old)
_create_cell_cellnid_new(domain.cells, domain.node_cellid, cell_cellnid_new)


In [66]:
for i in range(domain.nb_cells):
  a = cell_cellnid_old[i]
  b = cell_cellnid_new[i]
  a[:-1].sort()
  b[:-1].sort()
  try:
    np.testing.assert_array_equal(a, b)
  except AssertionError:
    print(i, a)
    print(b)
    break

[1, 2]